# 18. asyncio를 이용한 동시성

# Reminding

### Coroutine
Coroutine : 프로그램에서 순서는 일반적으로 불려지는 쪽(subroutine)이 부르는 쪽(main routine)에 속하고 있는 것이 대부분이지만 어느 쪽도 종속 관계가 아니라 대등한 관계로 서로 호출하는 것이다. 예를 들면, 게임 프로그램에서 각 플레이어 루틴은 서로 코루틴된다. 복수 프로세스 간에서 한정된 형태의 통신을 행하는 프로그램을 순차 제어로 실현한 것으로 볼 수도 있다.
<br>

__동작 과정__  
코루틴은 함수가 종료되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 돌아와서 코루틴의 코드를 실행. 코루틴이 종료된 것이 아니므로 코루틴의 내용도 계속 유지. 코루틴은 진입점이 여러 개인 함수.

__Back to 16.1__  
Generators vs Coroutines  

Python 2.5부터, generator에 .send(value) method가 추가되고, yield를 표현식 내에 사용할 수 있게 됨으로써, generator를 coroutine으로 사용할 수 있게 되었다.
쉽게 얘기하면, 코루틴은 제네레이터의 특별한 형태라고 생각하면 되는데, 제네레이터는 yield로 값을 발생시키지만, 코루틴은 yield로 값을 받아올 수 있다. 제너레이터는 next 함수를 반복 호출하여 값을 얻어내는 방식이고, 코루틴은 next 함수를 한 번만 호출(코루틴 안의 yield까지 코드 실행)한 뒤, send로 값을 주고 받는 방식.

Python Documentation에서는 [asyncio를 어떻게 설명](https://docs.python.org/ko/3/library/asyncio.html)하고 있을까?  
asyncio는 __async/await__ 구문을 사용하여 __동시성__ 코드를 작성하는 라이브러리입니다. asyncio는 고성능 네트워크 및 웹 서버, 데이터베이스 연결 라이브러리, 분산 작업 큐 등을 제공하는 여러 파이썬 비동기 프레임워크의 기반으로 사용됩니다. asyncio는 종종 I/O 병목이면서 고수준의 __구조화된__ 네트워크 코드에 가장 적합합니다. asyncio는 다음과 같은 작업을 위한 고수준 API 집합을 제공합니다:
- 파이썬 코루틴들을 동시에 실행하고 실행을 완전히 제어할 수 있습니다.
- 네트워크와 I/O와 IPC를 수행합니다.
- 자식 프로세스를 제어합니다.
- 큐를 통해 작업을 분산합니다.
- 동시성 코드를 동기화합니다.

## Index
- 18.1 스레드와 코루틴 비교
- 18.2 asyncio와 aiohttp로 내려받기
- 18.3 블로킹 호출을 에둘러 실행하기
- 18.4 asyncio 내려받기 스크립트 개선
- 18.5 콜백에서 Future와 코루틴으로
- 18.6 asyncio 서버 작성
- 18.7 요약
- 18.8 읽을거리


Concurrency is about dealing with lots of things at once.  
Parallelism is about doing lots of things at once.  
One is about structure, one is about execution.  

Parallelism은 그만큼의 코어가 있어야 한다. 하지만 Concurrency는 프로세스 갯수만큼 코어가 있지 않아도 가능하다! `asyncio`은 이벤트 루프에 기반한 코루틴으로 Concurrency(동시성)를 구현한 패키지이다. 이 챕터에서 하는 것은,</br>
- 스레드와 비동기 작업 간의 관계를 보여주면서 간단한 스레드 프로그램과 asyncio 동등 항목을 비교
- `asyncio.Future` 클래스와 `concurrent.futures.Future`가 어떻게 다른지
- Chapter 17의 플래그 다운로드 예제의 비동기 버전
- 스레드 또는 프로세스를 사용하지 않고 비동기 프로그래밍이 네트워크 애플리케이션에서 높은 동시성을 관리하는 방법
- 어떻게 코루틴이 비동기 프로그래밍에서 콜백에 비해 크게 개선되는지
- 차단 작업을 스레드 풀로 오프로드하여 이벤트 루프를 차단하지 않는 방법
- 비동기 서버 작성 및 동시성을 위해 웹 애플리케이션을 다시 생각하는 방법
- asyncio가 Python 생태계에 큰 영향을 미치는 이유

## 18.1 스레드와 코루틴 비교

In [3]:
#Example 18-1
import threading
import itertools
import time
import sys

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
def slow_function():
    #pretend waiting a long time for I/O
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

def main():
    result = supervisor()
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-8, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer: 42


Python에는 thread를 종료하는 API가 없으므로 메세지를 보내서 종료해야 한다. 여기서는 `signal.go`를 메세지로 사용. 이번에는 `@asyncio.coroutine`으로 어떻게 쓰는지?

In [1]:
#Example 18-2
import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
@asyncio.coroutine
def slow_function():
    # pretend waiting a long time for I/O
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

RuntimeError: This event loop is already running

spinner object: <Task pending coro=<spin() running at <ipython-input-1-b8d3fc9477dd>:6>>
| thinking/ thinking!

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: asyncio.async() function is deprecated, use ensure_future()


- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          

`time.sleep()`은 실행하는 동안 CPU가 유휴 상태로 들어가지만 `asyncio.sleep()`은 비동기적으로 실행하여 CPU가 다른 일을 할 수 있게 함.

In [6]:
#Example 18-3
def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

In [7]:
#Example 18-4
@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

SyntaxError: invalid syntax (<ipython-input-7-1a5c61053c86>, line 4)

두 구현 방식의 차이점! (디테일하게)
- `asyncio.Task`는 대략 `threading.Thread`와 동일하다.
- `Task`는 코루틴을 구동하고, `Thread`는 callable을 호출한다.
- `Task`를 직접 객체화하지 않고 `asyncio.async()`나 `loop.create_task()`에 코루틴을 넘겨줌으로 받는다.
- `Task` 객체를 받을 때, 그것은 이미 실행되기로 스케쥴되었다. `Thread` 인스턴스는 `start` 함수를 호출함으로 명시적으로 알려줘야 한다.
- threaded `supervisor`에서는, `slow_function`은 보통 함수이고 쓰레드에 의해 직접적으로 호출된다. `asyncio supervisor`에서는 코루틴이 `yield from`에 의해 구동된다.
- thread는 유효하지 않은 상태를 가지고 시스템을 떠나면서 어느 지점에서나 중단될 수 있기 때문에, 외부에서 중단할 수 있는 API가 없다. task는 `Task.cancel()`라는 코루틴 안에서 `CancelledError`를 발생시키는 인스턴스 메소드가 있다. 코루틴은 중단된 곳의 `yield`안에서 예외를 잡아냄으로 이것을 처리할 수 있다.
- `supervisor` 코루틴은 `loop.run_until_complete`와 함께 `main` 함수 안에서 반드시 실행되어야 한다.

<p><p>
    
코루틴을 사용하면 기본적으로 모든 것이 중단되지 않도록 보호됨. 프로그램의 나머지 부분을 실행하려면 명시적으로 yield 해야함. 멀티쓰레드의 동작을 동기화하기 위해 lock을 쓰기보다, 정의에 따라 '동기화'된 코루틴이 있음: 언제든지 하나만 실행됨. control을 포기하려면 `yield`나 `yield from`을 사용하면 스케쥴러로 컨트롤이 넘어감. 이것이 안전하게 코루틴을 종료할 수 있는 이유. 정의에 의하면 코루틴은 `yield` point에서 중단되었을 때만 종료가 가능하므로 `CancelledError` 예외를 처리하여 정리할 수 있음.

### asyncio.Future: Nonblocking by Design
`asyncio.Future`와 `concurrent.futures.Future`의 비교. 두 개는 비슷하게 동작하지만 교환 가능하지는 않다. `.done()`, `.add_done_callback()`, `.result()`를 가지는데, 앞에 두개는 동일하게 동작하지만 `.result()`은 다르게 동작함. 일단 timeout을 지정할 수 없다. `.result()`를 호출했을 때, future가 끝나지 않으면 결과를 기다리는 것을 차단하지 않는다. 대신, `InvalidStateError`를 발생시킨다.

`yield from`과 future를 사용하는 것은 이벤트 루프를 중단하지 않으면서 자동적으로 끝날 때까지 기다릴 수 있게 한다 - 왜냐하면, `asyncio`에서 `yield from`은 이벤트 루프로 컨트롤을 다시 돌려주는 것이기 때문에! 요약하면 `asyncio.Future`가 `yield from`과 같이 동작하도록 설계되었기 때문에 다음의 함수들은 필요없다. `my_future.add_done_callback()`, `my_future.result()`

### Yielding from Futrues, Tasks, and Coroutines
future와 coroutine의 관계: `yield from`해서 `asyncio.Future`에서 결과를 가져오기 때문.
`asyncio.async()` - coroutine과 future를 하나로 만듦.
`BaseEventLoop.create_task()` - 코루틴을 실행을 위해 스케줄하고 `asyncio.Task` 객체를 리턴한다. 

많은 `asyncio` 함수들이 코루틴을 받고 `asyncio`로 감싼다. 

In [ ]:
# Small Test
import asyncio
def run_sync(coro_or_future):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro_or_future)

run_sync(some_coroutine())

## 18.2 asyncio와 aiohttp로 내려받기
Python 3.4 기준으로 `asyncio`는 TCP와 UDP만 직접 지원함. 다른 프로토콜은 써드파티 패키지를 사용해야 함.

1. We start the process in `download_many` by feeding the event loop with several coroutine objects produced by calling `download_one`.
2. The `asyncio` event loop activates each coroutine in turn.
3. When a client coroutine such as `get_flag` uses `yield from` to delegate to a library coroutine—such as `aiohttp.request`—control goes back to the event loop, which can execute another previously scheduled coroutine.
4. The event loop uses low-level APIs based on callbacks to get notified when a blocking operation is completed.
5. When that happens, the main loop sends a result to the suspended coroutine.
6. The coroutine then advances to the next yield, for example, `yield from` `resp.read()` in `get_flag`. The event loop takes charge again. Steps 4, 5, and 6 repeat until the event loop is terminated.

In [3]:
import asyncio
import aiohttp
from flags import BASE_URL, save_flag, show, main

@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    image = yield from resp.read()
    return image

@asyncio.coroutine
def download_one(cc):
    image = yield from get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

@asyncio.coroutine
def download_many(cc_list):
    loop = asyncio.get_event_loop()
    to_do = [download_one(cc) for cc in sorted(cc_list)]
    wait_coro = asyncio.wait(to_do)
    res, _ = loop.run_until_complete(wait_coro)
    loop.close()
    return len(res)

if __name__ == '__main__':
    main(download_many)

ModuleNotFoundError: No module named 'flags'

`asyncio.wait()` 코루틴은 future나 coroutine를 받는다. `wait()`는 각각의 코루틴을 `Task`로 감싼다. 

`loop.run_until_complete` 함수는 future나 coroutine를 받아들인다. coroutine을 받으면, `run_until_complete`는 `wait`와 비슷하게 `Task`로 감싼다. Coroutine, future, task는 다 `yield from`을 기반으로 동작.

`asyncio`를 사용할 때 `next()`나 `send()`를 사용해서 coroutine chain을 사용하지 않고, `asyncio` 이벤트 루프가 대신한다.
가장 안쪽의 subgenerator는 실제로 I/O를 하는 라이브러리 함수

요약하면: 우리가 `asyncio`를 사용할 때 우리의 비동기적인 코드는 `asyncio` 자체에서 발생한 제너레이터를 대표하거나 `asyncio` 라이브러리 코루틴을 극도로 대표하는 코루틴으로 구성된다. 

## 18.3 블로킹 호출을 에둘러 실행하기
`blocking function`을 디스크나 네트워크 I/O를 하는 것들 중의 하나로 정의하고, 우리는 그것들을 `nonblocking function`처럼 취급할 수 없다고 주장함. 전체 애플리케이션의 진행을 중지하는 blocking call을 막는 두 가지 방법이 있다.
- blocking operation을 분리된 쓰레드에서 실행
- 모든 blocking operation을 nonblocking 비동기 호출로 전환

우리가 수천개의 연결을 다룬다면 각각의 연결마다 thread를 생성하는 것은 어려움. Callback은 적은 메모리 오버헤드로 비동기 호출을 구현할 수 있는 전통적인 방법. 응답을 기다리기보다 무언가 발생되면 호출될 함수를 등록함. 이렇게 하면, 우리가 하는 모든 호출이 nonblocking이 된다. 

이벤트 루프의 관점에서 콜백을 부르거나 중단된 코루틴에서 `.send()`를 부르는 것은 거의 동일하다. 각각의 중단된 코루틴에 대한 메모리 오버헤드가 있지만 각각의 스레드에 대한 오버헤드보다는 적다.
`flags_asyncio.py`가 `flags.py`가 5배 정도의 성능을 보이는 것이 말이 된다. `flags.py`는 각각의 다운로드를 하나씩 기다리는 곳에 수십 억개의 CPU 사이클을 사용한다. 반면에 `download_many`에서 `loop_until_complete`가 호출될 때, 이벤트 루프가 각각의 `download_one` 코루틴을 첫번째 `yield from`에 구동하고, 그러면 각 `get_flag` 코루틴을 첫 번째 `yield from`으로 구동하여 `aiohttp`를 호출한다. 이러한 호출은 차단되지 않기 때문에 모든 요청들은 빠르게 시작된다.

`asyncio` 인프라가 첫번째 응답을 받으면, 이벤트 루프는 대기중인 `get_flag` 코루틴으로 보냄. `get_flag`는 응답을 받아서 `resp.read()`를 호출하는 다음 `yield from`으로 진행하여 다시 메인 루프로 컨트롤을 넘겨준다. 각각의 `get_flag`가 리턴되면, 대리 제너레이터 `download_one`이 재개되고 파일을 저장한다.

순차적으로 하는 것보다 동시로 하는 것이 더 빠름. 600개의 HTTP 요청을 했을 때, 순차적으로 하는 것보다 `asyncio`를 사용하면 70배 정도 빨랐다고 한다...

## 18.4 asyncio 내려받기 스크립트 개선
`flags2_asyncio.py`의 구현

### Using asyncio.as_completed

In [7]:
#Example 18-7
import asyncio
import collections

import aiohttp
from aiohttp import web
import tqdm

from flags2_common import main, HTTPStatus, Result, save_flag

DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000

class FetchError(Exception):
    def __init__(self, country_code):
        self.country_code = country_code
        
@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    if resp.status == 200:
        image = yield from resp.read()
        return image
    elif resp.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.HttpProcessingError(
            code = resp.status, message=resp.reason,
            headers = resp.headers)
        
@asyncio.coroutine
#semaphore -> asyncio.Semaphore: limits the number of concurrent requests
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore):
            image = yield from get_flag(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'OK'
            
    if verbose and msg:
        print(cc, msg)
            
    return Result(status, cc)

앞에서는 `concur_req` 만큼의 스레드만 가지고 시작하는 방식이었다면, 여기서는 `download_coro` 함수가 생성하는 `asyncio.Semaphore`를 `download_one`에 전달하는 방식으로 동작한다. 세마포어는 `.aquire()`가 호출되면 내부 카운터가 감소, `.release()`가 호출되면 내부 카운터가 증가하는 방식. 초기 카운터는 매개변수로 전달되는 `concur_req` 값을 가짐. 0이 되면 blocking.  

`download_many`에 있던 많은 기능들이 이제 코루틴 `downloader_coro`에 있다. 

In [8]:
#Example 18-8
@asyncio.coroutine
def downloader_coro(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)
    to_do = [download_one(cc, base_url, semaphore, verbose)
             for cc in sorted(cc_list)]
    
    to_do_iter = asyncio.as_completed(to_do)
    if not verbose:
        to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))
    for future in to_do_iter:
        try:
            res = yield from future
        except FetchError as exc:
            country_code = exc.country_code
            try:
                error_msg = exc.__cause__.args[0]
            except IndexError:
                error_msg = exc.__cause__.__class__.__name__
            if verbose and error_msg:
                msg = '*** Error for {}: {}'
                print(msg.format(country_code, error_msg))
            status = HTTPStatus.error
        else:
            status = res.status
            
        counter[status] += 1
        
    return counter

def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)
    loop.close()
    
    return counts

if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)

TypeError: main() takes 0 positional arguments but 3 were given

달라진 점은 `requests`에서 `aiohttp`로 변경하면서 예외 처리의 달라짐과, `asyncio`를 사용하면서 변경된 점(`yield from` 등)이다.

### Using an Executor to Avoid Blocking the Event Loop
디스크 I/O에 사용되는 CPU 사이클이 많기 때문에 성능에 큰 중요한 영향을 미치게 된다. 17절의 방법에서는 많은 워커 스레드 중에 하나의 스레드가 중단되지만, 여기서 나오는 방법에서는 파일이 저장되는 동안 전체 어플리케이션이 중단되기 때문에 `run_in_executor` 방법을 사용한다.

In [ ]:
#Example 18-9
@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore):
            image = yield from get_flag(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exceptions as exc:
        raise FetchError(cc) from exc
    else:
        loop = asyncio.get_event_loop()
        loop.run_in_executor(None,
                            save_flag, image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'OK'
        
    if verbose and msg:
        print(cc, msg)
        
    return Result(status, cc)

## 18.5 콜백에서 Future와 코루틴으로
Callback? 어떤 일을 다른 객체에게 시키고, 그 일이 끝나는 것을 기다리는 것이 아니라 그 객체가 나를 다시 부를 때 까지 내 할일을 하고 있는 것. Nonblocking, asynchronous.

``` JavaScript
//Example 18-10
api_call1(request1, function (response1) {
    // stage 1
    var request2 = step1(response1);
    
    api_call2(request2, function (response2) {
        //stage 2
        var request3 = step2(response2);
        
        api_call3(request3, function (response3) {
            //stage 3
            step3(response3);
        });
    });
});
```

In [10]:
#Example 18-11 Callback hell in Python

def stage1(response1):
    request2 = step1(response1)
    api_call2(request2, stage2)
    
def stage2(response2):
    request3 = step2(response2)
    api_call3(request3, stage3)
    
def stage3(response3):
    step3(response3)
    
api_call1(request1, stage1)

NameError: name 'api_call1' is not defined

콜백 지옥에서 코루틴이 도움이 된다! 이벤트 루프 관점에서는 콜백과 거의 다를 것이 없지만 사용자 입장에서는 컨텍스트도 유지하고 하나의 함수 바디 안에 세 가지 동작을 집어넣을 수 있기 때문에 큰 변화가 있다.

In [11]:
#Example 18-12
@asyncio.coroutine
def three_stages(request1):
    response1 = yield from api_call1(request1)
    #stage 1
    request2 = step1(response1)
    response2 = yield from api_call2(request2)
    #stage 2
    request3 = step2(response2)
    response3 = yield from api_call3(request3)
    #stage 3
    step3(response3)
    
loop.create_task(three_stages(request1)) #must explicitly schedule execution

NameError: name 'loop' is not defined

callback 스타일과 달라진 점에 대해서 보면, 예외 처리를 하나의 예로 들 수 있다. callback 스타일은 비동기 호출이기 때문에 문맥이 보존되지 않아 바깥 쪽에서 예외처리를 할 수 없다. 그래서 하나의 비동기 호출에 두개의 콜백을 등록해서 하나는 성공한 결과, 하나는 에러 처리에 사용함. 딱 봐도 복잡..

하지만 18-12에서는 세 단계의 모든 비동기 호출이 같은 함수 안에 있기 때문에 예외를 일으키면, 각각 `try/except`안에 `yield from`을 집어넣음으로 처리할 수 있다.

일단은 `yield from`에 익숙해져야 한다. 일단 `yield from`을 적으면 일반 함수 처럼 쉽게 호출할 수도 없고, 코루틴의 실행을 명시적으로 스케줄하거나, 실행이 스케줄된 다른 코루틴에서 `yield from`을 통해 활성화해야한다. `loop.create_task(three_stages(request1))`의 호출이 없으면 아무일도 일어나지 않는다.

## 각각의 다운로드에 여러 개의 요청을 수행하기
threaded script에서는 쉽다. 하나 요청하고 다른 거 요청하고, 스레드 두번 중단하고, 지역 변수에 저장하고, 파일에 저장하면 됨.

콜백을 사용하면 문맥 보존이 되지 않기 때문에 저장이 까다로움.

코루틴을 사용하면 스레드 기반보다는 아니지만 그래도 좀 낫다.

In [13]:
#Example 18-13

@asyncio.coroutine
def http_def(url):
    res = yield from aiohttp.request('GET', url)
    if res.status == 200:
        ctype = res.headers.get('Content-type', '').lower()
        if 'json' in ctype or url.endswith('json'):
            data = yield from res.json()
        else:
            data = yield from res.read()
        return data
    
    elif res.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.errors.HttpProcessingError(
            code=res.status, message=res.reason,
            headers=res.headers)
        
@asyncio.coroutine
def get_country(base_url, cc):
    url = '{}/{cc}/metadata.json'.format(base_url, cc=cc.lower())
    metadata = yield from http_get(url)
    return metadata['country']

@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    return (yield from http_get(url))

@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore):
            image = yield from get_flag(base_url, cc)
        with (yield from semaphore):
            country = yield from get_country(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        country = country.replace(' ', '_')
        filename = '{}-{}.gif'.format(country, cc)
        loop = asyncio.get_event_loop()
        loop.run_in_executor(None, save_flag, image, filename)
        status = HTTPStatus.ok
        msg = 'OK'
        
    if verbose and msg:
        print(cc, msg)
        
    return Result(status, cc)

## 18.6 asyncio 서버 작성
처음에는 TCP, 나중에는 HTTP를 이용하여 유니코드 검색기를 만들어보자!
### An asyncio TCP Server
대부분의 내용들은 `charfinder.py`를 기반으로 한다.

In [ ]:
#Example 18-14 tcp_charfinder.py
import sys
import asyncio

from charfinder import UnicodeNameIndex

CLRF = b'\r\n'
PROMPT = b'?> '

index = UnicodeNameIndex()

@asyncio.coroutine
def handle_queries(reader, writer):
    while True:
        writer.write(PROMPT)
        yield from writer.drain()
        data = yield from reader.readline()
        try:
            query = data.decode().strip()
        except UnicodeDecodeError:
            query = '\x00'
        client = writer.get_extra_info('peername')
        print('Received from {}: {!r}'.format(client, query))
        if query:
            if ord(query[:1]) < 32:
                break
            lines = list(index.find_description_strs(query))
            if lines:
                writer.writelines(line.encode() + CRLF for line in lines)
            writer.write(index.status(query, len(lines)).encode() + CRLF)
            
            yield from writer.drain()
            print('Sent {} results'.format(len(lines)))
            
    print('Close the client socket')
    writer.close()

18-14에 있는 모든 I/O는 `bytes`이다. 네트워크에서 받을 때는 디코딩하고, 다시 보낼 때는 인코딩해야 한다. 한 가지 주의해야 할 점은, 일부 I/O 함수들은 `yield from`으로 실행해야 하는 반면에 다른 것들은 간단한 함수라는 것. 

In [14]:
#Example 18-15 tcp_charfinder.py

def main(address='127.0.0.1', port=2323):
    port = int(port)
    loop = asyncio.get_event_loop()
    server_coro = asyncio.start_server(handle_queries, address, port, loop=loop)
    server = loop.run_until_complete(server_coro)
    
    host = server.sockets[0].getsockname()
    print('Serving on {}. Hit CTRL-C to stop.'.format(host))
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        pass
    
    print('Server shutting down.')
    server.close()
    loop.run_until_complete(server.wait_close())
    loop.close()
    
if __name__ == '__main__':
    main(*sys.argv[1:])

ValueError: invalid literal for int() with base 10: '/root/.local/share/jupyter/runtime/kernel-5af35f37-c5ea-430e-bb4d-731ffec6b85a.json'

### An aiohttp Web Server

In [16]:
#Example 18-17 http_charfinder.py

@asyncio.coroutine
def init(loop, address, port):
    app = web.Application(loop=loop)
    app.router.add_route('GET', '/', home)
    handler = app.make_handler()
    server = yield from loop.create_server(handler, address, port)
    return server.sockets[0].getsockname()

def main(address="127.0.0.1", port=8888):
    port = int(port)
    loop = asyncio.get_event_loop()
    host = loop.run_until_complete(init(loop, address, port))
    print('Serving on {}. Hit CTRL-C to stop.'.format(host))
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        pass
    print('Server shutting down.')
    loop.close()
    
if __name__ == '__main__':
    main(*sys.argv[1:])

ValueError: invalid literal for int() with base 10: '/root/.local/share/jupyter/runtime/kernel-5af35f37-c5ea-430e-bb4d-731ffec6b85a.json'

TCP와의 비교  
```Python
server_coro = asyncio.start_server(handle_queries, address, port, loop=loop)
server = loop.run_until_complete(server_coro)
```
TCP에서는 서버가 만들어지고 실행이 main에서 스케줄하는 것이 이렇게 작동된다.

HTTP 예제에서는 `init` 함수가 서버를 이렇게 만든다.
```Python
server = yield from loop.create_server(handler, address, port)
```
하지만 `init` 자체도 코루틴이고 `main` 함수에서 실행시킨다.
```Python
host = loop.run_until_complete(init(loop, address, port))
```

In [15]:
#Example 18-18 http_charfinder.py

def home(request):
    query = request.GET.get('query', '').strip()
    print('Query: {!r}'.format(query))
    if query:
        descriptions = list(index.find_descriptions(query))
        res = '\n'.join(ROW_TPL.format(**vars(descr))
                       for descr in descriptions)
        msg = index.status(query, len(descriptions))
    else:
        descriptions = []
        res = ''
        msg = 'Enter words describing characters.'
        
    html = template.format(query=query, result=res, message=msg)
    print('Sending {} results'.format(len(descriptions)))
    return web.Response(content_type=CONTENT_TYPE, text=html)

`home`은 코루틴이 아니고 `yield from`이 없으면 그럴 필요도 없다.